# 📊 QEPC NBA Backtest Notebook - Adjusted for Max Accuracy

**Adjusted Version:** Added live API for recent games, quantum-inspired probability calibration (using PennyLane if installed), and use of 10-year data for walk-forward testing to reduce overfitting.

This notebook measures how well the QEPC NBA engine is performing over time.

It focuses on:
- 🗓️ Defining a backtest date range
- 🧮 Running `run_season_backtest(...)` to simulate all games in that range
- 📈 Summarizing accuracy & spread error
- 🧨 Inspecting the worst misses
- 🧹 Filtering to NBA–vs–NBA only (no exhibitions)
- 🌌 Quantum calibration for probs


## 🧩 1. Environment & Setup


In [ ]:
from notebook_context import *  # Setup
import pandas as pd
from nba_api.stats.endpoints import leaguegamelog  # Live API for recent games
import numpy as np
try:
    import pennylane as qml  # Quantum for calibration (install if needed)
except ImportError:
    print("PennyLane not installed—skip quantum calibration or install with !pip install pennylane")

# New: Fetch live recent games to add to backtest data
def add_live_games(historical_df, season='2025-26'):
    log = leaguegamelog.LeagueGameLog(season=season)
    live_df = log.get_data_frames()[0]
    combined = pd.concat([historical_df, live_df], ignore_index=True)
    print(f"Added {len(live_df)} live games to backtest.")
    return combined

# Load your 10-year data and add live
historical_df = pd.read_csv('data/merged_historical_data.csv')  # Your big file
historical_df = add_live_games(historical_df)

------

## 🎯 2. Backtest with Quantum Calibration

New: Added quantum-inspired calibration for win probs (using PennyLane if available) to handle uncertainties.


In [ ]:
# Define range
start_date_str = '2025-10-01'
end_date_str = '2025-11-29'

# Filter data
backtest_df = historical_df[(historical_df['gameDate'] >= start_date_str) & (historical_df['gameDate'] <= end_date_str)]

# Run your backtest function (assume exists)
results = run_season_backtest(backtest_df)  # Your function

# New: Quantum calibration if PennyLane installed
if 'qml' in globals():
    dev = qml.device("default.qubit", wires=1)
    @qml.qnode(dev)
    def calibrate_prob(prob):
        qml.RY(np.arctan(prob * 2 - 1), wires=0)  # Encode prob
        return qml.expval(qml.PauliX(wires=0))
    results['quantum_win_prob'] = results['Home_Win_Prob'].apply(lambda p: (calibrate_prob(p) + 1) / 2)
    print("Applied quantum calibration to probs.")

display(results.head())

## 📉 3. Analyze Results

New: Added Brier score for prob accuracy.


In [ ]:
# Your summary code...
# New: Brier score (measures prob quality)
from sklearn.metrics import brier_score_loss
brier = brier_score_loss(results['actual_home_win'], results['Home_Win_Prob'])
print(f"Brier Score (lower better): {brier:.3f} - Quantum calibrated: {brier_score_loss(results['actual_home_win'], results['quantum_win_prob']):.3f}")